## FILTRE SPAM PAR CLASSIFICATION NAIVE BAYESIENNE

### Introduction

L'objectif de ce projet est d'effectuer une classificaiton de messages sms comme étant du spam ou pas.
Je travaille sur le jeu de données mis à disposition par les auteurs T.A. Almeida et J.M.G Hidalgo sur le site [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) 

Il s'agit d'un jeu de données de 5572 SMS qui ont déjà été classés par les utilisateurs.

In [1]:
import pandas as pd
smsspamcollection = pd.read_csv('SMSSpamCollection', sep='\t', 
                                header=None, names=['Label', 'SMS'])
smsspamcollection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [2]:
smsspamcollection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
#environ 13% des messages est classé comme spam, donc 87% ne l'est pas
smsspamcollection['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

In [4]:
#réechantillonage simple du jeu de données entier
random_spamcollection = smsspamcollection.sample(frac=1, random_state=1)
#on place 80% du dataset dans le jeu d'entrainement soit 4458 sms, les 20% restant 
#constitueront le jeu de test 

# Calcul de l'index où séparer
index = round(len(random_spamcollection) * 0.8)

#Séparation
train_spamcollection = random_spamcollection[:index].reset_index(drop=True)
test_spamcollection = random_spamcollection[index:].reset_index(drop=True)

train_spamcollection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4458 entries, 0 to 4457
Data columns (total 2 columns):
Label    4458 non-null object
SMS      4458 non-null object
dtypes: object(2)
memory usage: 69.7+ KB


In [5]:
#je vérifie les proportions de spam/non-spam dans les jeux d'apprentissage et de test
#on est toujours sur du 87/13 dans les 2 cas - ok
train_spamcollection['Label'].value_counts(normalize=True)*100
test_spamcollection['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

#### Nettoyage et mise en forme du dataset

In [6]:
train_spamcollection.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
#Nettoyer et extraire les mots de chaque sms
def nettoyer_splitter(serie):
    pattern = r"\W"
    serie = serie.str.lower().replace(pattern, ' ')
    serie = serie.str.replace(pattern, ' ')
    serie = serie.str.split()
    return serie

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [ ]:
train_spamcollection['SMS'] = nettoyer_splitter(train_spamcollection['SMS'])
train_spamcollection.head()

In [8]:
#Construire le dictionnaire de mots du jeu de données
def vocabulaire(serie):
    vocabulaire = []
    for row in serie :
        for d in row :
            vocabulaire.append(d)
    ensemble = set(vocabulaire)
    vocabulaire = list(ensemble)
    return (vocabulaire)

7783

In [ ]:
vocabulaire = vocabulaire(train_spamcollection['SMS'])
vocabulaire[:10]

In [9]:
#Transformer le jeu de donnée en dataframe qui donne une vue de la fréquence d'occurence 
#de chaque mot dans l'ensemble des messages
def transformer_serie(serie, vocab):
    nbre_mots_par_msg = {mot_uniq: [0] * len(serie) for mot_uniq in vocab}

    for index, sms in enumerate(serie):
        for mot in sms:
            nbre_mots_par_msg[mot][index] += 1
    mots_par_msg = pd.DataFrame(nbre_mots_par_msg)    
    return (mots_par_msg)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4458 entries, 0 to 4457
Columns: 7783 entries, 0 to 鈥
dtypes: int64(7783)
memory usage: 264.7 MB


In [ ]:
test = transformer_serie(train_spamcollection['SMS'],vocabulaire)
test.info()

In [10]:
trans_spamcollection = pd.concat([train_spamcollection,test], axis=1)
trans_spamcollection.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


#### Calcul des constantes de notre algorithme
Nous utilisons ici l'algorithme de classification naive de Bayes. 
Les constantes sont **la probabilité d'avoir un message spam**, **celle d'avoir un message non spam**, **le nombre de mots appartenant à la catégorie spam**, **le nombre de mots appartenant à la catégorie non-spam**, **le nombre de mots du vocabulaire** et **la constante de Laplace** 

In [19]:
# Calcul probabilité de spam/non-spam
def proba_spam(df, col, cat) :
    categorie = df[df[col]==cat]
    proba = categorie.shape[0]/df.shape[0]
    return(proba)

In [21]:
p_spam = proba_spam(trans_spamcollection, "Label", "spam")
p_spam

0.13458950201884254

In [22]:
p_non_spam = proba_spam(trans_spamcollection, "Label", "ham")
p_non_spam

0.8654104979811574

In [28]:
# Calcul du nombre de mots tombant dans chaque catégorie
def nombre_mot_ds_cat(df, col_label, col_sms, cat) :
    categorie = df[df[col_label]==cat]
    nbre = 0
    for sms in categorie[col_sms]:
        nbre += len(sms) 
    return(nbre)
 
n_vocabulaire = len(vocabulaire)

In [29]:
n_spam = nombre_mot_ds_cat(trans_spamcollection, "Label", "SMS", "spam")
n_spam

15190

In [34]:
n_non_spam = nombre_mot_ds_cat(trans_spamcollection, "Label", "SMS", "ham")
n_non_spam

57237

In [31]:
n_vocabulaire

7783

In [32]:
# Initialisation du terme de lissage (lissage de Laplace)
alpha = 1

#### Calcul des variables de notre algorithme
Ce sont les probabilités conditionnelles des différents mots apparaissant 
dans chaque catégorie (spam et non spam)

In [36]:
# Construction de deux dictionnaires dict_spam et dict_non_spam contenant 
# respectivement pour chaque clé la probabilité d'avoir le mot du vocabulaire 
# correspondant sachant que le message est un spam
# et la probabilité d'avoir le mot du vocabulaire 
# correspondant sachant que le message n'est pas un spam

def dictionnaire_cat(df, col_label, cat, vocab, liste_const):
    #le paramètre liste_const contient les constantes calculées dans les cellules 
    #précédentes : list_const[0] = alpha, list_const[1] = n_vocabulaire
    # et list_const[2] = n_spam ou n_non_spam selon les cas
    dict_cat = {mot_uniq: 0 for mot_uniq in vocab}
    categorie = df[df[col_label]==cat]
    for mot in vocab :
        n_mot_sachant_cat = categorie[mot].sum()
        p_mot_sachant_cat = (n_mot_sachant_cat+liste_const[0])/(liste_const[2]+(liste_const[0]*liste_const[1]))
        dict_cat[mot] = p_mot_sachant_cat
    return (dict_cat)

In [37]:
liste_constantes_spam = [alpha, n_vocabulaire, n_spam]
liste_constantes_non_spam = [alpha, n_vocabulaire, n_non_spam]

dict_spam = dictionnaire_cat(trans_spamcollection, "Label", "spam", vocabulaire, liste_constantes_spam)
dict_non_spam = dictionnaire_cat(trans_spamcollection, "Label", "ham", vocabulaire, liste_constantes_non_spam)


In [39]:
dict_spam['yes']

0.0007399991294127889

In [40]:
dict_spam['mumhas']

4.3529360553693465e-05

In [41]:
dict_non_spam['yes']

0.0010150722854506305

In [42]:
dict_non_spam['mumhas']

3.075976622577668e-05